# Regression Phase 11: multitasking with Lasso

In [2]:
import sklearn
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools
import numpy as np
from sklearn.linear_model import MultiTaskLasso, Lasso
import graphlab

In [3]:
sklearn.__version__

'0.17.1'

In [4]:
print (__doc__)

Automatically created module for IPython interactive environment


In [5]:
def get_numpy_data(data_sframe, features, output):
    data_sframe['constant'] = 1 #this is how to add a constant to an SFrame
    # add the column 'constant' to the front of the features list so that we can extract it along with the ohters:
    features = ['constant'] +features # this is how we combine two lists
    # select the columns of data_SFrame given by the features list into the SFrame features_sframe (now including constant):
    features_sframe = data_sframe[features]
    # the following line will convert the features_SFrame into a numpy matrix:
    feature_matrix = features_sframe.to_numpy()
    # assign the column of data_sframe associated with the output to the SArray output_sarray
    output_sarray = data_sframe[output]
    # the following will convert the SArray into a numpy array by first converting it to a list
    output_array = output_sarray.to_numpy()
    return(feature_matrix, output_array)

In [6]:
hotels_NY = graphlab.SFrame('NY.csv') 
hotels_NY['price'] = hotels_NY['price'].astype(float)
hotels_NY['rates'] = hotels_NY['rates'].astype(float)
hotels_NY['zipcode'] = hotels_NY['zipcode'].astype(float)
hotels_NY['guests'] = hotels_NY['guests'].astype(float)
hotels_NY = hotels_NY[hotels_NY['price'] > 10]

hotels_Boston = graphlab.SFrame('Boston.csv') 
hotels_Boston['price'] = hotels_Boston['price'].astype(float)
hotels_Boston['rates'] = hotels_Boston['rates'].astype(float)
hotels_Boston['zipcode'] = hotels_Boston['zipcode'].astype(float)
hotels_Boston['guests'] = hotels_Boston['guests'].astype(float)
hotels_Boston = hotels_Boston[hotels_Boston['price'] > 10]

hotels_Chicago = graphlab.SFrame('Chicago.csv') 
hotels_Chicago['price'] = hotels_Chicago['price'].astype(float)
hotels_Chicago['rates'] = hotels_Chicago['rates'].astype(float)
hotels_Chicago['zipcode'] = hotels_Chicago['zipcode'].astype(float)
hotels_Chicago['guests'] = hotels_Chicago['guests'].astype(float)
hotels_Chicago = hotels_Chicago[hotels_Chicago['price'] > 10]

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1492928057.log


This non-commercial license of GraphLab Create for academic use is assigned to dchen@albany.edu and will expire on November 05, 2017.


Finished parsing file /Users/Peter/NY.csv

Parsing completed. Parsed 100 lines in 0.231489 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,float,float,str,str,str,str,int,int,str,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


...

Finished parsing file /Users/Peter/Chicago.csv

Parsing completed. Parsed 100 lines in 0.075344 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[str,str,int,float,float,str,str,str,str,int,int,str,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Peter/Chicago.csv

Parsing completed. Parsed 7051 lines in 0.048227 secs.

In [7]:
hotels_new = hotels_NY.join(hotels_Boston, how = 'outer')
hotels_new = hotels_new.join(hotels_Chicago, how = 'outer')

In [8]:
from math import log, sqrt
hotels_new['size_sqrt'] = hotels_new['size'].apply(sqrt)
hotels_new['star_squared'] = hotels_new['star']*hotels_new['star']
hotels_new['rates_rating'] = hotels_new['rates']*hotels_new['rating']
# drop na values
# sf_filter = sf[(sf['carrier'] == 'US')]
#hotels = hotels[hotels['name'] != 'FieldHouse Jones']
hotels_new

name,zone,zipcode,star,rating,rates,checkin,checkout
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Courtyard New YorkDowntown Manhattan/World ...,Wall Street - FinancialDistrict ...,10006.0,3.0,4.3,46.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
Park Lane Hotel,Central Park,10019.0,4.0,4.0,6.0,04/21/2017,04/22/2017
The Belvedere Hotel,Broadway - Times Square,10036.0,3.5,4.2,16.0,04/21/2017,04/22/2017


In [9]:
all_features = ['star', 
                'star_squared',
                'size_sqrt',
                'rates_rating', 
                'zipcode',
                'rating',
                'rates',
                'size',
                'guests'
               ]
hotels_new = hotels_new[hotels_new['star'] != None]

model_all = graphlab.linear_regression.create(hotels_new, target='price', features = all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=5e6)

Linear regression:

--------------------------------------------------------

Number of examples          : 38496

Number of features          : 9

Number of unpacked features : 9

...

| 5         | 6        | 0.000002  | 1.468760     | 704.710726         | 152.937683    |

| 6         | 7        | 0.000002  | 1.509031     | 708.418668         | 153.779560    |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [10]:
# get the non-zero weights
non_zero_weight = model_all["coefficients"][model_all["coefficients"]["value"] > 0]
non_zero_weight.print_rows(num_rows=10)

+--------------+-------+-----------------+--------+
|     name     | index |      value      | stderr |
+--------------+-------+-----------------+--------+
| (intercept)  |  None |  148.617685217  |  None  |
|     star     |  None |  7.82253762889  |  None  |
| star_squared |  None |   3.5379148204  |  None  |
|  size_sqrt   |  None |  0.786830224227 |  None  |
|    rating    |  None |  1.94124863158  |  None  |
|     size     |  None | 0.0612675962455 |  None  |
+--------------+-------+-----------------+--------+
[6 rows x 4 columns]



In [11]:
(training_and_validation, testing) = hotels_new.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validat

In [12]:
import pprint
#set a result set for the validation rss
validation_rss = {}
for l1_penalty in np.logspace(1,7, num=13):
    #here we taking all the features into consideration
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residual = validation['price'] - predictions
    rss = sum(residual**2)
    #dictionary is key value pair thing
    validation_rss[l1_penalty] = rss

pprint.pprint(validation_rss)
#here we can use the min to print out the smallest value in a dictionary very important
print min(validation_rss.items(), key = lambda x:x[1])

{10.0: 268492056.8316335,
 31.622776601683793: 268492755.02004635,
 100.0: 268494963.0753368,
 316.22776601683796: 268501947.4546832,
 1000.0: 268524052.96246976,
 3162.2776601683795: 268594146.3043443,
 10000.0: 268817696.84467596,
 31622.776601683792: 269544396.0402416,
 100000.0: 272050974.4316129,
 316227.76601683791: 281788261.0603819,
 1000000.0: 334767644.0565404,
 3162277.6601683795: 459416115.29744786,
 10000000.0: 479835446.8600876}
(10.0, 268492056.8316335)


In [13]:
model_best = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose = False,
                                              l2_penalty=0., l1_penalty=10.0)
non_zero_weight_best = model_best["coefficients"][model_best["coefficients"]["value"] > 0]
print model_best["coefficients"]["value"].nnz()
non_zero_weight_best.print_rows(num_rows=20)

10
+--------------+-------+------------------+--------+
|     name     | index |      value       | stderr |
+--------------+-------+------------------+--------+
| (intercept)  |  None |  32.6830731076   |  None  |
|     star     |  None |  16.9631426983   |  None  |
| star_squared |  None |  5.90000718909   |  None  |
|  size_sqrt   |  None |  2.56628443176   |  None  |
| rates_rating |  None | 0.00258532834924 |  None  |
|    rating    |  None |  10.4220820743   |  None  |
|     size     |  None |  0.154454766112  |  None  |
|    guests    |  None |   8.595618513    |  None  |
+--------------+-------+------------------+--------+
[8 rows x 4 columns]



In [39]:
max_nonzeros = 6

In [40]:
l1_penalty_values = np.logspace(5, 7, num=20)

In [41]:
coef_dict ={}
for l1_penalty in l1_penalty_values:
    model  = graphlab.linear_regression.create(training,target = 'price', features = all_features, validation_set = None,
                                             verbose = None, l2_penalty=0.,l1_penalty=l1_penalty)
    coef_dict[l1_penalty] = model['coefficients']['value'].nnz()
    
pprint.pprint(coef_dict)

{100000.0: 10,
 127427.49857031347: 10,
 162377.67391887208: 10,
 206913.80811147901: 8,
 263665.08987303555: 8,
 335981.82862837811: 8,
 428133.23987193959: 8,
 545559.47811685142: 8,
 695192.79617756058: 8,
 885866.79041008325: 8,
 1128837.8916846884: 7,
 1438449.888287663: 7,
 1832980.7108324375: 6,
 2335721.4690901213: 6,
 2976351.441631319: 5,
 3792690.1907322537: 3,
 4832930.2385717519: 2,
 6158482.110660254: 1,
 7847599.7035146067: 1,
 10000000.0: 1}


In [42]:
l1_penalty_min = 1438449.888287663
l1_penalty_max = 2976351.441631319

In [43]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

In [44]:
validation_rss = {}
for l1_penalty in l1_penalty_values:
    model = graphlab.linear_regression.create(training,target='price', features=all_features,
                                             validation_set = None, verbose = False, l2_penalty= 0, l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    residuals = predictions - validation['price']
    rss = sum(residuals**2)
    validation_rss[l1_penalty] = rss, model['coefficients']['value'].nnz()
    
validation_rss

{1438449.888287663: (354852863.3395697, 7),
 1519392.0753057501: (359139027.25768495, 7),
 1600334.2623238373: (363297290.8222947, 6),
 1681276.4493419244: (366642762.65656614, 6),
 1762218.6363600115: (370043917.0666373, 6),
 1843160.8233780987: (373502728.33559275, 6),
 1924103.010396186: (377020882.8720901, 6),
 2005045.1974142732: (380598092.80847305, 6),
 2085987.3844323603: (385078165.6126793, 6),
 2166929.5714504477: (390721861.37658316, 6),
 2247871.7584685348: (396547821.5613216, 6),
 2328813.9454866219: (402556037.5919883, 6),
 2409756.1325047091: (408746509.4685857, 6),
 2490698.3195227962: (415118320.2756001, 6),
 2571640.5065408833: (421672265.4725574, 6),
 2652582.6935589705: (428490820.41652113, 6),
 2733524.8805770576: (433942347.9058434, 5),
 2814467.0675951447: (438610423.7824869, 5),
 2895409.2546132319: (443344133.5903675, 5),
 2976351.441631319: (448143463.0128483, 5)}

In [45]:
bestRSS = 9223372036854775807.0
for k,v in validation_rss.iteritems():    
    if (v[1] == max_nonzeros) and (v[0] < bestRSS):
        bestRSS = v[0]
        bestl1 = k
        
print bestRSS, bestl1

363297290.822 1600334.26232


In [46]:
model = graphlab.linear_regression.create(training,target='price',features= all_features,
                                         validation_set = None, verbose = False,
                                         l2_penalty=0., l1_penalty=1600334)


In [47]:
non_zero_weight_test = model['coefficients'][model['coefficients']['value']>0]
non_zero_weight_test.print_rows(num_rows=20)
model['coefficients'].print_rows(num_rows=20)

+--------------+-------+-----------------+--------+
|     name     | index |      value      | stderr |
+--------------+-------+-----------------+--------+
| (intercept)  |  None |  114.386439457  |  None  |
|     star     |  None |   10.226270841  |  None  |
| star_squared |  None |  4.22230289753  |  None  |
|  size_sqrt   |  None |  1.23436188373  |  None  |
|    rating    |  None |  3.98505518439  |  None  |
|     size     |  None | 0.0872451686419 |  None  |
+--------------+-------+-----------------+--------+
[6 rows x 4 columns]

+--------------+-------+-----------------+--------+
|     name     | index |      value      | stderr |
+--------------+-------+-----------------+--------+
| (intercept)  |  None |  114.386439457  |  None  |
|     star     |  None |   10.226270841  |  None  |
| star_squared |  None |  4.22230289753  |  None  |
|  size_sqrt   |  None |  1.23436188373  |  None  |
| rates_rating |  None |       0.0       |  None  |
|   zipcode    |  None |       0.0       |

In [48]:
predictions = model.predict(testing)
residuals = predictions - testing['price']
rss = sum(residuals**2)
print rss
print len(predictions)

83961839.5408
3874


In [49]:
# use the same l1_penalty
# single task performance
hotels_NY['size_sqrt'] = hotels_NY['size'].apply(sqrt)
hotels_NY['star_squared'] = hotels_NY['star']*hotels_NY['star']
hotels_NY['rates_rating'] = hotels_NY['rates']*hotels_NY['rating']

model_NY = graphlab.linear_regression.create(hotels_NY, target='price', features = all_features,
                                              validation_set=None, 
                                             
                                              l2_penalty=0., l1_penalty=1600334)

predictions = model_NY.predict(testing)
residuals = predictions - testing['price']
rss = sum(residuals**2)
print rss
print len(predictions)

Linear regression:

--------------------------------------------------------

Number of examples          : 25144

Number of features          : 9

Number of unpacked features : 9

...

| 6         | 7        | 0.000002  | 0.489442     | 641.871332         | 140.806832    |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

89776044.2737
3874


As we can see, the model using the multi-tasking performs better on the testing data than the single task based on the RSS